In [1]:
library(cowplot)
library(infer)
library(repr)
library(tidyverse)
library(broom)
library(digest)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
salmon <- read.csv("~/stat-201/Stat201-group-project-4/ASL_master.csv")
head(salmon)

,Species,Length.Measurement.Type,sampleYear,ASLProjectType,LocationID,sampleDate,Length,Weight,Sex,Salt.Water.Age,⋯,Flag,Gear,SASAP.Region,LocationUnique,DistrictID,Sub.DistrictID,Stat.area,Lat,Lon,AWC_CODE
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>
1,chinook,length not taken,1992,commercial catch,Affleck Canal/Spanish Is/Louise Cove,1992-03-31,NA,NA,examined but did not identify,3,⋯,NA,troll,Southeast,Affleck Canal/Spanish Is/Louise Cove-commercial catch-10510,105,10,10510,NA,NA,NA
2,chinook,length not taken,1992,commercial catch,Affleck Canal/Spanish Is/Louise Cove,1992-03-31,NA,NA,examined but did not identify,3,⋯,NA,troll,Southeast,Affleck Canal/Spanish Is/Louise Cove-commercial catch-10510,105,10,10510,NA,NA,NA
3,chinook,length not taken,1992,commercial catch,Affleck Canal/Spanish Is/Louise Cove,1992-03-31,NA,NA,examined but did not identify,3,⋯,NA,troll,Southeast,Affleck Canal/Spanish Is/Louise Cove-commercial catch-10510,105,10,10510,NA,NA,NA
4,chinook,length not taken,1992,commercial catch,Affleck Canal/Spanish Is/Louise Cove,1992-03-31,NA,NA,examined but did not identify,3,⋯,NA,troll,Southeast,Affleck Canal/Spanish Is/Louise Cove-commercial catch-10510,105,10,10510,NA,NA,NA
5,chinook,length not taken,1992,commercial catch,Affleck Canal/Spanish Is/Louise Cove,1992-03-31,NA,NA,examined but did not identify,4,⋯,NA,troll,Southeast,Affleck Canal/Spanish Is/Louise Cove-commercial catch-10510,105,10,10510,NA,NA,NA
6,chinook,length not taken,1992,commercial catch,Affleck Canal/Spanish Is/Louise Cove,1992-03-31,NA,NA,examined but did not identify,3,⋯,NA,troll,Southeast,Affleck Canal/Spanish Is/Louise Cove-commercial catch-10510,105,10,10510,NA,NA,NA
